In [1]:
import os
from pathlib import Path
import pandas as pd

from sklearn.datasets import load_diabetes

from datahandles import preprocess_numeric
from utils import Config

from tabllm import load_dataset, load_and_preprocess_dataset
from datasets import load_from_disk

import numpy as np

/afs/glue.umd.edu/home/glue/p/a/pasand/home/miniconda3/envs/transhyper2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
ds_name = 'jungle'
ds_raw = load_dataset(dataset_name=ds_name, data_dir=Path(f'./tabllm/data/datasets/{ds_name}'))
print(ds_raw)
print(ds_raw.columns)

       white_piece0_strength  white_piece0_file  white_piece0_rank  \
0                        0.0                1.0                8.0   
1                        0.0                2.0                8.0   
2                        0.0                4.0                8.0   
3                        0.0                5.0                8.0   
4                        0.0                6.0                8.0   
...                      ...                ...                ...   
44814                    7.0                6.0                1.0   
44815                    7.0                0.0                0.0   
44816                    7.0                1.0                0.0   
44817                    7.0                2.0                0.0   
44818                    7.0                4.0                0.0   

       black_piece0_strength  black_piece0_file  black_piece0_rank  label  
0                        0.0                0.0                8.0   True  
1      

In [12]:
all_cols = ['white_piece0_strength', 'white_piece0_file', 'white_piece0_rank',
       'black_piece0_strength', 'black_piece0_file', 'black_piece0_rank']
categorical = []
numerical = list(set(all_cols) - set(categorical) - set(['label']))

print(categorical)
print(numerical)

[]
['white_piece0_rank', 'white_piece0_file', 'black_piece0_strength', 'black_piece0_rank', 'white_piece0_strength', 'black_piece0_file']


In [2]:
ds_name = 'income'
ds_raw = load_and_preprocess_dataset(dataset_name=ds_name, data_dir=Path(f'./tabllm/data/datasets/{ds_name}'))
# print(ds_raw)
ds_txt = load_from_disk(f"tabllm/data/datasets_serialized/{ds_name}")
ds_txt = pd.DataFrame(ds_txt) # type: ignore
val_counts = ds_raw['label'].value_counts()
print(val_counts)
print('Max const. predictor acc:', max(val_counts)/sum(val_counts) * 100)

label
False    37155
True     11687
Name: count, dtype: int64
Max const. predictor acc: 76.07182343065395


In [3]:
ds_txt

,note,label
0,The Age is 39. The Race is White. The Sex is M...,False
1,The Age is 54. The Race is White. The Sex is M...,True
2,The Age is 32. The Race is White. The Sex is M...,True
3,The Age is 45. The Race is White. The Sex is M...,False
4,The Age is 60. The Race is Black. The Sex is M...,False
...,...,...
48837,The Age is 39. The Race is White. The Sex is F...,False
48838,The Age is 64. The Race is Black. The Sex is M...,False
48839,The Age is 38. The Race is White. The Sex is M...,False
48840,The Age is 44. The Race is Asian-Pac-Islander....,False


In [8]:
ds_name = "income"
# ds = load_from_disk(f"tabllm/data/data_ser/{ds_name}")
ds = load_dataset(dataset_name=ds_name, data_dir=Path(f"./tabllm/data/datasets/{ds_name}"))
print(ds)

       age         workclass     education      marital_status  \
16465   39  Self-emp-not-inc          11th  Married-civ-spouse   
5625    54  Self-emp-not-inc     Bachelors  Married-civ-spouse   
30273   32           Private       HS-grad  Married-civ-spouse   
3136    45  Self-emp-not-inc  Some-college       Never-married   
4521    60           Private          10th  Married-civ-spouse   
...    ...               ...           ...                 ...   
32511   25         Local-gov     Bachelors       Never-married   
5192    32           Private     Bachelors  Married-civ-spouse   
12172   27           Private     Bachelors       Never-married   
235     59         State-gov       HS-grad  Married-civ-spouse   
29733   33           Private     Bachelors  Married-civ-spouse   

              occupation   relationship                race     sex  \
16465   Transport-moving        Husband               White    Male   
5625     Exec-managerial        Husband               White    Ma

In [3]:
from transformers import BertModel, BertTokenizer

model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [12]:
for name, param in model.named_parameters():
    print(name)
    # if "classifier" not in name: # Assuming the last layer is named "classifier"
    #     param.requires_grad = False

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [5]:
from utils import Config, ConfigObject
from datahandles import TabLLMDataObject, CombinedTabLLMTextDataset
import yaml
from jinja2 import Environment, FileSystemLoader, select_autoescape
import os

cfg = Config("./cfgs/bert.yaml")
tabllm_do = TabLLMDataObject(cfg, set_hyponet_in_dim=True)
test_ds = tabllm_do.data['test']
print(test_ds[0]['shots'])

print(len(tokenizer.tokenize(test_ds[0]['shots'])))

TabLLMDataObject initialized with 4 training set(s),1 validation set(s), and 1 test set(s).
Maximum number of features across all datasets: 103
Hyponet in_dim set to max number of features (=103)
[(748, 748), (45211, 45211), (20640, 20640), (48842, 48842)]
Example: The median income is 3.0375. The median age is 26. The total rooms is 1912. The total bedrooms is 339. The population is 1002. The households is 311. The latitude is 36.37. The longitude is -119.55.

Is this house block valuable? Yes or no? Answer:
|||
No
 Example: The median income is 3.4419. The median age is 15. The total rooms is 7626. The total bedrooms is 1570. The population is 3823. The households is 1415. The latitude is 33.87. The longitude is -117.60.

Is this house block valuable? Yes or no? Answer:
|||
No
 Example: The median income is 4.4762. The median age is 30. The total rooms is 5398. The total bedrooms is 926. The population is 2672. The households is 864. The latitude is 33.94. The longitude is -117.55.



In [5]:
if "decoder" not in "66.decoder.manam":
    print(True)
else:
    print(False)

False


In [3]:
from datetime import datetime
print(datetime.now().strftime("%y%m%d%H%M"))

2507281250


In [6]:
import pyarrow as pa

# Open the stream and read all record batches into a Table
with open('./tabllm/data/datasets_serialized/creditg/dataset.arrow', 'rb') as f:
    reader = pa.ipc.open_stream(f)
    table = reader.read_all()

# Convert to a pandas DataFrame (optional)
df = table.to_pandas()
df.head()

,note,label
0,The Status of existing checking account is < 0...,True
1,The Status of existing checking account is 0 <...,False
2,The Status of existing checking account is no ...,True
3,The Status of existing checking account is < 0...,True
4,The Status of existing checking account is < 0...,False


In [2]:
a = [2, 1]
a.extend([i for i in range(5) if i not in a])

In [3]:
a

[2, 1, 0, 3, 4]